### **Approach 1: TF-IDF Method**

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np
nlp = English()
nlp.add_pipe('sentencizer')


def summarizer(text, tokenizer, max_sent_in_summary=4):
    # Create spacy document for further sentence level tokenization
    doc = nlp(text_corpus.replace("\n", ""))
    # Use 'text' attribute instead of 'string' to extract sentence text
    sentences = [sent.text.strip() for sent in doc.sents]
    # Let's create an organizer which will store the sentence ordering to later reorganize the
    # scored sentences in their correct order
    sentence_organizer = {k:v for v,k in enumerate(sentences)}
    # Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
    tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None,
                                        strip_accents='unicode',
                                        analyzer='word',
                                        token_pattern=r'\w{1,}',
                                        ngram_range=(1, 3),
                                        use_idf=1,smooth_idf=1,
                                        sublinear_tf=1,
                                        stop_words = 'english')
    # Passing our sentences treating each as one document to TF-IDF vectorizer
    tf_idf_vectorizer.fit(sentences)
    # Transforming our sentences to TF-IDF vectors
    sentence_vectors = tf_idf_vectorizer.transform(sentences)
    # Getting sentence scores for each sentences
    sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()
    # Getting top-n sentences
    N = max_sent_in_summary
    top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]
    # Let's now do the sentence ordering using our prebaked sentence_organizer
    # Let's map the scored sentences with their indexes
    mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
    # Ordering our top-n sentences in their original ordering
    mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
    ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]
    # Our final summary
    summary = " ".join(ordered_scored_sentences)
    return summary

In [2]:
text_corpus = """
His debut was in place of Sachin Tendulkar and Virender Sehwag and he followed MS Dhoni as captain.

Since then, he has been incomparable.

Kohli made his ICC Men’s Cricket World Cup debut in 2011 as the second youngest member of India’s victorious squad at just 22 – though he had already made 45 ODI appearances by the start of the tournament, such was the regard with which he was held in.

His India debut did not foreshadow what was to come, Kohli dismissed for just 12 against Sri Lanka, but it did not take him long to make an impact and he brought up a maiden half-century in his fourth game.

A first hundred came over a year later but the Delhi native had to be patient and wait for the absence of some Indian greats to nail down a place in the side.

Kohli came in for a rested Tendulkar during a tri-nation ODI tournament at the start of 2010, seizing his opportunity with another century.

Away from the pitch, Kohli was something of a conundrum but for his captain at the time, Dhoni, the team liked Kohli just the way he was.

He said: "It's important for him to be himself. You have to show off what you are and he has matured now.

“To us, he comes as a humble guy. He might come across differently to the world.

"Being a good human being is important, but it can come later, what's important now is that he [Kohli] scores on the field.”

Kohli ended the year as India’s leading run-scorer in ODIs with an average of 47.38 and with it came a spot in the 2011 World Cup squad on home soil.

He set the tone with a century in the tournament’s opening game before rebuilding with Gambhir in the final following the early departures of Sehwag and Tendulkar, making 35 to help India on the path to an historic triumph.

From there, a Test debut came and emboldened by a brief stint as ODI captain in 2013, scoring a century in just his second match as skipper, Kohli began to break records.

Against Australia, he hit the fastest ODI century by an Indian and fastest against the Aussies, reaching three figures from just 52 balls.

A first century as Test captain followed in 2014, Australia again the opponents, while standing in for Dhoni before taking over the reins full-time mid-way through the series.

Captaincy requires the shouldering of expectations, the weight of the nation rests with you and when the nation totals hundreds of millions of people, the weight could be unbearable.

But for Kohli, it was a spark for some of the greatest years of his career.

In 2016 and 2017, he scored a double century across four consecutive Test series, overtaking a record held by Don Bradman and Rahul Dravid.

No one could keep up with Kohli. He was awarded the Sir Garfield Sobers Trophy for ICC Men’s Cricketer of the Year in 2017, repeating the feat the next year.

His trophy cabinet was further filled in 2018, with ICC Men's Test Player of the Year and ICC Men's ODI Player of the Year gongs, also being named one of Time's 100 Most Influential People.
"""
print("Summarizer Result: \n", summarizer(text=text_corpus, tokenizer=nlp, max_sent_in_summary=3))


Summarizer Result: 
 Kohli made his ICC Men’s Cricket World Cup debut in 2011 as the second youngest member of India’s victorious squad at just 22 – though he had already made 45 ODI appearances by the start of the tournament, such was the regard with which he was held in. ”Kohli ended the year as India’s leading run-scorer in ODIs with an average of 47.38 and with it came a spot in the 2011 World Cup squad on home soil. His trophy cabinet was further filled in 2018, with ICC Men's Test Player of the Year and ICC Men's ODI Player of the Year gongs, also being named one of Time's 100 Most Influential People.


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


### **Approach 2 : LSA Method**

In [3]:
from sklearn.decomposition import TruncatedSVD

def summarizer_lsa(text, tokenizer, max_sent_in_summary=4):
    # Create spacy document for further sentence level tokenization
    doc = nlp(text.replace("\n", ""))
    sentences = [sent.text.strip() for sent in doc.sents]
    sentence_organizer = {k: v for v, k in enumerate(sentences)}

    tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None,
                                        strip_accents='unicode',
                                        analyzer='word',
                                        token_pattern=r'\w{1,}',
                                        ngram_range=(1, 3),
                                        stop_words = 'english')
    tf_idf_matrix = tf_idf_vectorizer.fit_transform(sentences)

    lsa_model = TruncatedSVD(n_components=1, random_state=42)
    lsa_matrix = lsa_model.fit_transform(tf_idf_matrix)

    sentence_scores = np.array(lsa_matrix).ravel()

    N = max_sent_in_summary
    top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]

    mapped_top_n_sentences = [(sentence, sentence_organizer[sentence]) for sentence in top_n_sentences]
    mapped_top_n_sentences = sorted(mapped_top_n_sentences, key=lambda x: x[1])
    ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

    summary = " ".join(ordered_scored_sentences)
    return summary


In [4]:
text_corpus = """
His debut was in place of Sachin Tendulkar and Virender Sehwag and he followed MS Dhoni as captain.

Since then, he has been incomparable.

Kohli made his ICC Men’s Cricket World Cup debut in 2011 as the second youngest member of India’s victorious squad at just 22 – though he had already made 45 ODI appearances by the start of the tournament, such was the regard with which he was held in.

His India debut did not foreshadow what was to come, Kohli dismissed for just 12 against Sri Lanka, but it did not take him long to make an impact and he brought up a maiden half-century in his fourth game.

A first hundred came over a year later but the Delhi native had to be patient and wait for the absence of some Indian greats to nail down a place in the side.

Kohli came in for a rested Tendulkar during a tri-nation ODI tournament at the start of 2010, seizing his opportunity with another century.

Away from the pitch, Kohli was something of a conundrum but for his captain at the time, Dhoni, the team liked Kohli just the way he was.

He said: "It's important for him to be himself. You have to show off what you are and he has matured now.

“To us, he comes as a humble guy. He might come across differently to the world.

"Being a good human being is important, but it can come later, what's important now is that he [Kohli] scores on the field.”

Kohli ended the year as India’s leading run-scorer in ODIs with an average of 47.38 and with it came a spot in the 2011 World Cup squad on home soil.

He set the tone with a century in the tournament’s opening game before rebuilding with Gambhir in the final following the early departures of Sehwag and Tendulkar, making 35 to help India on the path to an historic triumph.

From there, a Test debut came and emboldened by a brief stint as ODI captain in 2013, scoring a century in just his second match as skipper, Kohli began to break records.

Against Australia, he hit the fastest ODI century by an Indian and fastest against the Aussies, reaching three figures from just 52 balls.

A first century as Test captain followed in 2014, Australia again the opponents, while standing in for Dhoni before taking over the reins full-time mid-way through the series.

Captaincy requires the shouldering of expectations, the weight of the nation rests with you and when the nation totals hundreds of millions of people, the weight could be unbearable.

But for Kohli, it was a spark for some of the greatest years of his career.

In 2016 and 2017, he scored a double century across four consecutive Test series, overtaking a record held by Don Bradman and Rahul Dravid.

No one could keep up with Kohli. He was awarded the Sir Garfield Sobers Trophy for ICC Men’s Cricketer of the Year in 2017, repeating the feat the next year.

His trophy cabinet was further filled in 2018, with ICC Men's Test Player of the Year and ICC Men's ODI Player of the Year gongs, also being named one of Time's 100 Most Influential People.
"""
print("Summarizer Result: \n", summarizer_lsa(text=text_corpus, tokenizer=nlp, max_sent_in_summary=3))


Summarizer Result: 
 From there, a Test debut came and emboldened by a brief stint as ODI captain in 2013, scoring a century in just his second match as skipper, Kohli began to break records. But for Kohli, it was a spark for some of the greatest years of his career. No one could keep up with Kohli.


### **Approach 3 : Frequency Count Method**

In [5]:
from collections import Counter

def summarizer_frequency(text, tokenizer, max_sent_in_summary=4):
    doc = nlp(text.replace("\n", ""))
    sentences = [sent.text.strip() for sent in doc.sents]
    sentence_organizer = {k: v for v, k in enumerate(sentences)}

    # Tokenize words and filter stopwords
    words = [token.text for token in doc if not token.is_stop and not token.is_punct]
    word_frequencies = Counter(words)
    max_frequency = max(word_frequencies.values())
    word_frequencies = {word: freq / max_frequency for word, freq in word_frequencies.items()}

    # Score sentences based on word frequencies
    sentence_scores = []
    for sentence in sentences:
        score = 0
        for word in sentence.split():
            if word in word_frequencies:
                score += word_frequencies[word]
        sentence_scores.append(score)

    N = max_sent_in_summary
    top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores)[::-1][:N]]

    mapped_top_n_sentences = [(sentence, sentence_organizer[sentence]) for sentence in top_n_sentences]
    mapped_top_n_sentences = sorted(mapped_top_n_sentences, key=lambda x: x[1])
    ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

    summary = " ".join(ordered_scored_sentences)
    return summary


In [6]:
text_corpus = """
His debut was in place of Sachin Tendulkar and Virender Sehwag and he followed MS Dhoni as captain.

Since then, he has been incomparable.

Kohli made his ICC Men’s Cricket World Cup debut in 2011 as the second youngest member of India’s victorious squad at just 22 – though he had already made 45 ODI appearances by the start of the tournament, such was the regard with which he was held in.

His India debut did not foreshadow what was to come, Kohli dismissed for just 12 against Sri Lanka, but it did not take him long to make an impact and he brought up a maiden half-century in his fourth game.

A first hundred came over a year later but the Delhi native had to be patient and wait for the absence of some Indian greats to nail down a place in the side.

Kohli came in for a rested Tendulkar during a tri-nation ODI tournament at the start of 2010, seizing his opportunity with another century.

Away from the pitch, Kohli was something of a conundrum but for his captain at the time, Dhoni, the team liked Kohli just the way he was.

He said: "It's important for him to be himself. You have to show off what you are and he has matured now.

“To us, he comes as a humble guy. He might come across differently to the world.

"Being a good human being is important, but it can come later, what's important now is that he [Kohli] scores on the field.”

Kohli ended the year as India’s leading run-scorer in ODIs with an average of 47.38 and with it came a spot in the 2011 World Cup squad on home soil.

He set the tone with a century in the tournament’s opening game before rebuilding with Gambhir in the final following the early departures of Sehwag and Tendulkar, making 35 to help India on the path to an historic triumph.

From there, a Test debut came and emboldened by a brief stint as ODI captain in 2013, scoring a century in just his second match as skipper, Kohli began to break records.

Against Australia, he hit the fastest ODI century by an Indian and fastest against the Aussies, reaching three figures from just 52 balls.

A first century as Test captain followed in 2014, Australia again the opponents, while standing in for Dhoni before taking over the reins full-time mid-way through the series.

Captaincy requires the shouldering of expectations, the weight of the nation rests with you and when the nation totals hundreds of millions of people, the weight could be unbearable.

But for Kohli, it was a spark for some of the greatest years of his career.

In 2016 and 2017, he scored a double century across four consecutive Test series, overtaking a record held by Don Bradman and Rahul Dravid.

No one could keep up with Kohli. He was awarded the Sir Garfield Sobers Trophy for ICC Men’s Cricketer of the Year in 2017, repeating the feat the next year.

His trophy cabinet was further filled in 2018, with ICC Men's Test Player of the Year and ICC Men's ODI Player of the Year gongs, also being named one of Time's 100 Most Influential People.
"""
print("Summarizer Result: \n", summarizer_frequency(text=text_corpus, tokenizer=nlp, max_sent_in_summary=3))


Summarizer Result: 
 Kohli made his ICC Men’s Cricket World Cup debut in 2011 as the second youngest member of India’s victorious squad at just 22 – though he had already made 45 ODI appearances by the start of the tournament, such was the regard with which he was held in. From there, a Test debut came and emboldened by a brief stint as ODI captain in 2013, scoring a century in just his second match as skipper, Kohli began to break records. His trophy cabinet was further filled in 2018, with ICC Men's Test Player of the Year and ICC Men's ODI Player of the Year gongs, also being named one of Time's 100 Most Influential People.
